# システム統合１（有限オートマトン）
それでは、これまでに実装してきたモジュールを統合して音声対話システムを作成してみましょう。

- 仕様
    - 音声認識：Google（ストリーミング、オリジナルVAD）
    - 音声合成：Google
    - 言語理解：ルールベース（フレーム）
    - 対話管理：有限オートマトン

In [1]:
# 必要なラブラリを読み込む
# from __future__ import division

# import re
# import sys, os

# import numpy as np
# import math
# import struct

# import pyaudio
# from six.moves import queue

# from pydub import AudioSegment
# from pydub.playback import play

# # Google音声認識を使用するためのライブラリ
# from google.cloud import speech

# 実装したライブラリを読み込む
#from asr_google_streaming import GoogleStreamingASR, MicrophoneStream       # VADなし
from asr_google_streaming_vad import GoogleStreamingASR, MicrophoneStream   # VADあり
from tts_google import GoogleTextToSpeech
from dm_fst import DmFst
from slu_rule import SluRule

In [2]:
# 音声認識クラスのパラメータ
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

# 音声合成の初期化
tts = GoogleTextToSpeech()

# 言語理解の初期化
slu_parser = SluRule()

In [3]:
# 対話管理の初期化
dm = DmFst()

# 初期状態の発話
system_utterance = dm.get_system_utterance()
tts.generate(system_utterance)
print("システム： " + system_utterance)
tts.play()

# 対話が終了状態に移るまで対話を続ける
while(dm.end == False):

    # 音声認識入力を得る	
    micStream = MicrophoneStream(RATE, CHUNK)
    asrStream = GoogleStreamingASR(RATE, micStream)
    print('<<<please speak>>>')
    result_asr = asrStream.get_asr_result()

    if hasattr(result_asr, 'alternatives') == False:
        print('Invalid ASR input')
        continue

    result_asr_utterance = result_asr.alternatives[0].transcript
    print("ユーザ： " + result_asr_utterance)
    
    # 言語理解
    result_slu = slu_parser.parse_frame(result_asr_utterance)
    print(result_slu)
    
    # 対話管理へ入力
    system_utterance = dm.enter(result_slu)
    tts.generate(system_utterance)
    print("システム： " + system_utterance)
    tts.play()
    
    print()


システム： こんにちは。京都レストラン案内です。どの地域のレストランをお探しですか。
<<<please speak>>>
音声パワー： 48.6[dB] 途中結果: 百万遍でお願いしますユーザ： 百万遍でお願いします
[{'intent': None, 'slot_name': 'place', 'slot_value': '百万遍'}]
システム： どのような料理がお好みですか。

<<<please speak>>>
音声パワー： 46.5[dB] ユーザ： 中華料理が食べたいい
[{'intent': None, 'slot_name': 'genre', 'slot_value': '中華'}]
システム： ご予算はおいくらぐらいですか。

<<<please speak>>>
音声パワー： 47.5[dB] ユーザ： 2000円以下でお願いしますす
[{'intent': None, 'slot_name': 'budget', 'slot_value': '2000円'}]
システム： 検索します。

